# CLASIFICACIÓN DE EMOCIONES UTILIZANDO BASE DE DATOS DEAP

## Organización de las bases de datos

In [2]:
#Librerías Necesarias Para el Código organizadas por tipo de llamado y orden alfabético.
import scipy
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pickle
import pywt
from scipy import signal
from sklearn import svm
import statistics as stats
from sklearn.decomposition import PCA
from scipy.signal import butter, lfilter
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score as acc
from sklearn.utils.class_weight import compute_class_weight

# Loading EEG and labels dataset made by the code organizer
File= open('Wavelets_Excombatants/EEG.pckl', 'rb')
EEG = pickle.load(File)
File.close()

File= open('Wavelets_Excombatants/y.pckl', 'rb')
label_matrix= pickle.load(File)
File.close()

# Dimensions
num_valences= 3
num_channels= 62
num_trials= 50
num_rois = 8


In [3]:
label_matrix.shape
print(EEG.shape)

(3, 50, 62, 128)


## USANDO DWT

## So, according to the nyquist theorem, and noted the sample rate fixed in 128 Hz the sub bands will be:

#### 32 to 64
#### 16 to 32
#### 8 to 16
#### 4 to 8
#### 2 to 4

## But if we consider that brain rhythms are::

#### Gamma: (30, 45)
#### Beta: (12, 30)
#### Alpha: (8, 12)
#### Theta: (4, 8)
#### Delta: (0, 4)


## Then, to capture and separate most of the brain rhythms, we can use only
#### 32 to 64     >>> Gamma
#### 16 to 32     >>> High Beta
#### 8 to 16     >>> Alpha
#### 4 to 8    >>> Theta
#### 2 to 4    >>> Delta


## Which mean that we need 5 levels of decomposition for DWT:

In [8]:
# Function plot_wavelet is unused
def plot_wavelet(DWT_values,_values_f,subj,trial):
    for n, c_D in enumerate(DWT_values):
        len_D = len(DWT_values)
        print(c_D.shape)
        print(len_D)
        fft_vals = np.absolute(np.fft.rfft(c_D[subj,trial]))
        plt.figure()
        plt.plot(fft_vals[1:-1])
        if n ==(0):
            plt.title("cA_{0} ,{1}".format(n, _values_f[n]))
        else:
            plt.title("cD_{0}, {1}".format(n+1, _values_f[n]))
        plt.xlabel("Instances")
        plt.ylabel("Energy")
    plt.show()

# Function characterization is used to create region of interest (ROI) clustering channels by indexes, then 
# we return several vectors according to the experiment, i.e. if the experiment has valences or not, if it will
# combine information of rois or bands or even both with features, etc.

def characterization(Filtered_Data,num_valences,num_trials,num_channels,wave,kind):
    #Wavelet transform calculation, it is specified the dataset to transform, wavelet type and decomposition level
    DWT_values= pywt.wavedec(Filtered_Data,wave,mode='sp1', level=4)
    DWT_values = DWT_values[0:4]
    len_D = len(DWT_values)
    
    ROI_1 = [0, 1, 2, 3, 4, 8, 9, 10, 18]
    ROI_2 = [5, 6, 7, 14, 15, 16, 17]
    ROI_3 = [11, 12, 13, 19, 20, 21, 22]
    ROI_4 = [23, 24, 32, 33, 41, 42, 50]
    ROI_5 = [30, 31, 39, 40, 48, 49, 56]
    ROI_6 = [25, 26, 27, 28, 29, 34, 35, 36, 37, 38]
    ROI_7 = [43, 44, 45, 46, 47, 51, 52, 53, 54, 55]
    ROI_8 = [57, 58, 59, 60, 61]
    ROIS = [ROI_1, ROI_2, ROI_3, ROI_4, ROI_5, ROI_6, ROI_7, ROI_8] #vector of regions of interest, each index is a channel
    
    ROI_ALL=[[]]*8
    for i in range(8):
        ROI_ALL[i] = [[]]*(len_D)
    #Features_Wavelet store the data transform dividing into EEG Rhythms
    Features_Stats=np.zeros([num_valences,len(DWT_values),num_trials,len(ROIS),8])
    #print(len(ROI_ALL))
    for roi in range(len(ROIS)):
        for band in range(len(DWT_values)):
            ROI_ALL[roi][band]=(np.mean(DWT_values[band][:,:,ROIS[roi],:],axis=2)) # we compute the mean of all channels of the same ROI
            #Statistical Features
            Features_Stats[:,band,:,roi,0] = np.max(ROI_ALL[roi][band],axis=2) # max value feature
            Features_Stats[:,band,:,roi,1] = np.min(ROI_ALL[roi][band],axis=2) # min value feature
            Features_Stats[:,band,:,roi,2] = np.mean(ROI_ALL[roi][band],axis=2) # mean
            Features_Stats[:,band,:,roi,3] = np.var(ROI_ALL[roi][band],axis=2) # variance
            Features_Stats[:,band,:,roi,4] = np.std(ROI_ALL[roi][band],axis=2) # standard deviation
            Features_Stats[:,band,:,roi,5] = np.median(ROI_ALL[roi][band],axis=2) # median
            Features_Stats[:,band,:,roi,6] = kurtosis(ROI_ALL[roi][band],axis=2) # kurtosis
            Features_Stats[:,band,:,roi,7] = skew(ROI_ALL[roi][band],axis=2) #skewness
            
    #in this part, we organize the dimensions to later take to the classifier
    Features_Stats_ROI = np.swapaxes(Features_Stats,1,2)
    Features_Stats_ROI = np.swapaxes(Features_Stats_ROI,2,3)
    Features_Stats_ROI = Features_Stats_ROI.reshape(num_valences,num_trials,len_D*len(ROIS),8)
    Features_Stats_ROI = np.swapaxes(Features_Stats_ROI,1,2)
    Features_Stats_Bands = Features_Stats.reshape(num_valences,len_D,num_trials,len(ROIS)*8)
    Features_Stats_Rois = Features_Stats.reshape(num_valences,len(ROIS),num_trials,len_D*8)
    
    Features_Stats_ValenceBands = np.swapaxes(Features_Stats,0,1)
    Features_Stats_ValenceBands = np.swapaxes(Features_Stats_ValenceBands,1,2)
    Features_Stats_ValenceBands = Features_Stats_ValenceBands.reshape(len_D,num_trials,num_valences,len(ROIS)*8)
    Features_Stats_ValenceBands = Features_Stats_ValenceBands.reshape(len_D,num_trials,num_valences*len(ROIS)*8)
    
    Features_Stats_All = np.swapaxes(Features_Stats_ValenceBands,0,1)
    #Features_Stats_ALL = np.swapaxes(Features_Stats_ALL,2,3)
    Features_Stats_All = Features_Stats_All.reshape(num_trials,num_valences*len(ROIS)*8*len(DWT_values))
    Features_Stats_ValencesRois = np.swapaxes(Features_Stats,0,2)
    Features_Stats_ValencesRois = Features_Stats_ValencesRois.reshape(num_trials,len(DWT_values),num_valences*len(ROIS)*8)
    Features_Stats_ValencesRois = np.swapaxes(Features_Stats_ValencesRois,0,1)
    
    Features_Stats_Valencesbands = np.swapaxes(Features_Stats,0,2)
    Features_Stats_Valencesbands = Features_Stats_Valencesbands.reshape(num_trials,len(ROIS),num_valences*len(DWT_values)*8)
    Features_Stats_Valencesbands = np.swapaxes(Features_Stats_Valencesbands,0,1)
  
    
    Features_Stats_Valences =  np.swapaxes(Features_Stats,0,3)
    Features_Stats_Valences = np.swapaxes(Features_Stats_ROI,1,3)
    Features_Stats_Valences = Features_Stats_Valences.reshape(len(ROIS)*len_D,num_trials,num_valences*8)
    
    Features_Stats_original = Features_Stats
    Features_Stats=Features_Stats.reshape(num_valences,len_D,num_trials,64)
    
    # Return the vector according to the input    
    if kind == 'complete' or kind == 0:
        return Features_Stats
    elif kind == 'ROI' or kind ==1:
        return Features_Stats_ROI
    elif kind == 'all' or kind ==2:
        return Features_Stats_All
    elif kind == "valroi" or kind ==3:
        return Features_Stats_ValencesRois
    elif kind == "valences" or kind ==4:
        return Features_Stats_Valences
    elif kind == "original" or kind == 5:
        return Features_Stats_original
    elif kind == 'bands' or kind ==6:
        return Features_Stats_Bands
    elif kind == "rois" or kind ==7:
        return Features_Stats_Rois
    elif kind == "valencesbans" or kind ==8:
        return Features_Stats_Valencesbands
                                                              
    else:
        raise NameError('Invalid kind of output')

# Unused
def graficar(F1_test,emocion):  
    Bandas=np.arange(1,6)
    contador=1
    Emocion=['Valence','Arousal','Dominance','Liking']
    for s in range(0,32):
        plt.figure(contador,figsize=(6,5))
        plt.title('Sujeto de estudio: '+str(s+1)+'  Emocion: '+Emocion[emocion])
        plt.bar(Bandas, F1_test[s,emocion,:], align='center', alpha=0.5, color=['black', 'red', 'green', 'blue', 'cyan'])
        plt.ylim(0,1)
        plt.xlabel('Frequency Band')
        plt.ylabel('F1 Score')
        plt.show()
        contador+=1

# This funtion is used to find the hyperparameter c and gamma
def svc_param_selection(X, y, nfolds,Gamma):
    Cs = 1.**np.arange(0,1)
    #gammas = 2.**np.arange(-8,8)
    param_grid = {'C':Cs}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf',gamma=Gamma,class_weight = 'balanced'),param_grid,  cv=nfolds)
   # print(y)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_




In [ ]:
a = np.array([[2, 4, 6], [1, 3, 5]])
print(a)
b = np.reshape(a,6)
b

In [ ]:
Feature_Stats_1=characterization(EEG,3,50,62,'sym8',1)
print(Feature_Stats_1.shape)

In [ ]:
flatten=Features_Stats_1=characterization(EEG,3,50,62,'sym8',1)
flatten = np.swapaxes(flatten, 1,2)
unflatten=Features_Stats_1=characterization(EEG,3,50,62,'sym8',5)
unflatten = np.swapaxes(unflatten,1,2)
#unflatten = np.swapaxes(unflatten,2,3)

print("f",flatten.shape)
print("u",unflatten.shape)



In [ ]:
print(unflatten[0,0,1,1,:])
print(flatten[0,0,5,:])


In [ ]:
unflatten_2.shape

In [ ]:
flatten_2 = flatten
num_valences = 3
num_bands = 4
num_rois = 8
unflatten_2 = np.zeros([num_valences,50,num_bands,num_rois])

for valence in range(3):
    for i in range(0,8):
        for j in range(0,4):
            unflatten_2[valence,0,j,i] = flatten_2[valence,0,j+i*num_bands,3]
            

In [ ]:
unflatten_2.shape

In [ ]:
print(unflatten_2[0,0,0,2])
print(flatten_2[0,0,8,3])


In [ ]:
print(flatten_2[0,0,0,:])
print(unflatten_2[0,0])

In [ ]:
unflatten[0,0,:,:,0].shape
unflatten

# SVM KERNEL RBF

## Classification using 3 valences and 40 bands: 
### Here it was performed a classification for each valence and it was combined the bands and ROIs in a single dimension. 

In [7]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold as KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------

Valences = 3
N_Band= 32
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],1)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,0,:,:]))
    F1_test_fold = np.zeros([N_folds,Valences,N_Band])
    ACC_test_fold = np.zeros([N_folds,Valences,N_Band])
    F1_test = np.zeros([Valences,N_Band])
    ACC_test = np.zeros([Valences,N_Band])
    STD_F1_test = np.zeros([Valences,N_Band])
    STD_ACC_test = np.zeros([Valences,N_Band])
    #SVM_Prediction_test=np.zeros([Valences,N_folds,N_Band])
    
    
    for valence in range(Valences):
        y_test = []
        print('Analizando Wavelet: '+wave_name[wave]+' Valencia : '+str(valence))

        fold = 0
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[valence,1,:,:]),label_matrix):
            ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
            #print(emo,sub,fold,test_index)
            #print(ytest.shape,y_test.shape)
            y_test.append(ytest)
                        
            for band in range(0,N_Band):        
                Xtrain,Xtest = np.squeeze(Features_Stats[valence,band,train_index,:]), (Features_Stats[valence,band,test_index,:])
                scaler = MinMaxScaler();
                Xtrain = scaler.fit_transform(Xtrain)
                Xtest = scaler.transform(Xtest)
                Xtrain = normalize(Xtrain)
                Xtest = normalize(Xtest)

                Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                Median= stats.median(Distances)
                Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
                SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
                SVM_Classifier.fit(Xtrain,ytrain)
                #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
                SVM_Prediction_test= SVM_Classifier.predict(Xtest)
                F1_test_fold[fold,valence,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
                ACC_test_fold[fold,valence,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
                #print(y_test)
            fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        
        for band in range(0,N_Band):
            #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
            F1_test[valence,band] = np.mean(F1_test_fold[:,valence,band])
            ACC_test[valence,band]= np.mean(ACC_test_fold[:,valence,band])
            STD_F1_test[valence,band] = np.std(F1_test_fold[:,valence,band])
            STD_ACC_test[valence,band] = np.std(ACC_test_fold[:,valence,band])
        print(F1_test)
        

    wav= wave_name[wave]+'_valences_'+'bandrois'
    
    #Pickle save F1_Score
    
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 

                
                

Analizando Wavelet: sym8 Valencia : 0
INICIO TEST
[[0.46666667 0.48       0.41333333 0.32       0.53333333 0.6
  0.52       0.57333333 0.48       0.58666667 0.14666667 0.68
  0.61333333 0.45333333 0.65333333 0.64       0.28       0.53333333
  0.61333333 0.50666667 0.49333333 0.41333333 0.48       0.42666667
  0.53333333 0.62666667 0.44       0.50666667 0.48       0.46666667
  0.44       0.49333333]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.       

## Classification using 3 valences and 5 bands:
### Here it was performed a classification for each valence and it was combined the rois and statistical featues in a single dimension.

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------

Valences =3
N_Band=5
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],6)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,0,:,:]))
    F1_test_fold = np.zeros([N_folds,Valences,N_Band])
    ACC_test_fold = np.zeros([N_folds,Valences,N_Band])
    F1_test = np.zeros([Valences,N_Band])
    ACC_test = np.zeros([Valences,N_Band])
    STD_F1_test = np.zeros([Valences,N_Band])
    STD_ACC_test = np.zeros([Valences,N_Band])
    
    for valence in range(Valences):
        y_test = []
        print('Analizando Wavelet: '+wave_name[wave]+' Valencia : '+str(valence))

        fold = 0
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[valence,1,:,:])):
            ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
            #print(emo,sub,fold,test_index)
            #print(ytest.shape,y_test.shape)
            y_test.append(ytest)
                        
            for band in range(0,N_Band):        
                Xtrain,Xtest = np.squeeze(Features_Stats[valence,band,train_index,:]), (Features_Stats[valence,band,test_index,:])
                scaler = MinMaxScaler();
                Xtrain = scaler.fit_transform(Xtrain)
                Xtest = scaler.transform(Xtest)
                Xtrain = normalize(Xtrain)
                Xtest = normalize(Xtest)

                Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                Median= stats.median(Distances)
                Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
                SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
                SVM_Classifier.fit(Xtrain,ytrain)
                #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
                SVM_Prediction_test= SVM_Classifier.predict(Xtest)
                F1_test_fold[fold,valence,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
                ACC_test_fold[fold,valence,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
                #print(y_test)
            fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        for band in range(0,N_Band):
            #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
            F1_test[valence,band] = np.mean(F1_test_fold[:,valence,band])
            ACC_test[valence,band]= np.mean(ACC_test_fold[:,valence,band])
            STD_F1_test[valence,band] = np.std(F1_test_fold[:,valence,band])
            STD_ACC_test[valence,band] = np.std(ACC_test_fold[:,valence,band])
        print(F1_test,ACC_test,STD_F1_test)
        

    wav= wave_name[wave]+'_valences_'+'bands'
    
    #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 

                
                

## Classification using 3 valences and 8 rois:
### Here it was performed a classification for each valence and it was combined bands and statistical featues in a single dimension.

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------

Valences =3
N_Band=8
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],7)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,0,:,:]))
    F1_test_fold = np.zeros([N_folds,Valences,N_Band])
    ACC_test_fold = np.zeros([N_folds,Valences,N_Band])
    F1_test = np.zeros([Valences,N_Band])
    ACC_test = np.zeros([Valences,N_Band])
    STD_F1_test = np.zeros([Valences,N_Band])
    STD_ACC_test = np.zeros([Valences,N_Band])
    
    for valence in range(Valences):
        y_test = []
        print('Analizando Wavelet: '+wave_name[wave]+' Valencia : '+str(valence))

        fold = 0
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[valence,1,:,:])):
            ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
            #print(emo,sub,fold,test_index)
            #print(ytest.shape,y_test.shape)
            y_test.append(ytest)
                        
            for band in range(0,N_Band):        
                Xtrain,Xtest = np.squeeze(Features_Stats[valence,band,train_index,:]), (Features_Stats[valence,band,test_index,:])
                scaler = MinMaxScaler();
                Xtrain = scaler.fit_transform(Xtrain)
                Xtest = scaler.transform(Xtest)
                Xtrain = normalize(Xtrain)
                Xtest = normalize(Xtest)

                Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                Median= stats.median(Distances)
                Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
                SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
                SVM_Classifier.fit(Xtrain,ytrain)
                #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
                SVM_Prediction_test= SVM_Classifier.predict(Xtest)
                F1_test_fold[fold,valence,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
                ACC_test_fold[fold,valence,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
                #print(y_test)
            fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        for band in range(0,N_Band):
            #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
            F1_test[valence,band] = np.mean(F1_test_fold[:,valence,band])
            ACC_test[valence,band]= np.mean(ACC_test_fold[:,valence,band])
            STD_F1_test[valence,band] = np.std(F1_test_fold[:,valence,band])
            STD_ACC_test[valence,band] = np.std(ACC_test_fold[:,valence,band])
        print(F1_test,ACC_test,STD_F1_test)
        

    wav= wave_name[wave]+'_valences_'+'rois'
    
    #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 

                
                

## Classification using 40 bands and no-valences:
### Here it was classified combining the bands and ROIs in a single dimension, and valences with statistical features in another dimension

In [ ]:
Features_Stats.shape

In [11]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------

N_Band=32
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],4)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,:,:]))
    F1_test_fold = np.zeros([N_folds,N_Band])
    ACC_test_fold = np.zeros([N_folds,N_Band])
    F1_test = np.zeros([N_Band])
    ACC_test = np.zeros([N_Band])
    STD_F1_test = np.zeros([N_Band])
    STD_ACC_test = np.zeros([N_Band])
    y_test = []
    print('Analizando Wavelet: '+wave_name[wave])

    fold = 0
    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        #print(emo,sub,fold,test_index)
        #print(ytest.shape,y_test.shape)
        y_test.append(ytest)

        for band in range(0,N_Band):        
            Xtrain,Xtest = np.squeeze(Features_Stats[band,train_index,:]), (Features_Stats[band,test_index,:])
            scaler = MinMaxScaler();
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            Xtrain = normalize(Xtrain)
            Xtest = normalize(Xtest)

            Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            Median= stats.median(Distances)
            Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
            Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
            SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
            SVM_Classifier.fit(Xtrain,ytrain)
            #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
            SVM_Prediction_test= SVM_Classifier.predict(Xtest)
            F1_test_fold[fold,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
            ACC_test_fold[fold,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
            #print(y_test)
        fold+=1
    #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
    print('INICIO TEST')
    #Calcular F1_test
    #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    for band in range(0,N_Band):
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
        F1_test[band] = np.mean(F1_test_fold[:,band])
        ACC_test[band]= np.mean(ACC_test_fold[:,band])
        STD_F1_test[band] = np.std(F1_test_fold[:,band])
        STD_ACC_test[band] = np.std(ACC_test_fold[:,band])
    print(F1_test,ACC_test,STD_F1_test)
        

    wav= wave_name[wave]+'_bandrois'
    
    #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 
                
                

Analizando Wavelet: sym8
INICIO TEST
[0.45333333 0.50666667 0.92       0.44       0.45333333 0.68
 0.94666667 0.48       0.44       0.66666667 0.92       0.34666667
 0.52       0.70666667 0.93333333 0.49333333 0.50666667 0.66666667
 0.94666667 0.56       0.4        0.70666667 0.94666667 0.36
 0.41333333 0.70666667 0.94666667 0.34666667 0.52       0.68
 0.93333333 0.41333333] [0.5  0.62 0.9  0.46 0.46 0.7  0.94 0.5  0.48 0.72 0.9  0.3  0.46 0.72
 0.92 0.48 0.5  0.7  0.96 0.58 0.44 0.72 0.96 0.34 0.44 0.72 0.96 0.34
 0.58 0.72 0.94 0.46] [0.42037021 0.4909854  0.21664102 0.43939352 0.42037021 0.46647615
 0.20396078 0.41182521 0.43939352 0.43204938 0.21664102 0.37094474
 0.41182521 0.44542115 0.21081851 0.42290004 0.40133112 0.46188022
 0.20396078 0.43939352 0.43204938 0.44542115 0.20396078 0.4207929
 0.41397799 0.44542115 0.20396078 0.40529824 0.44301994 0.46647615
 0.21081851 0.38040914]
(32,)
Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏
(32, 50, 24)
Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏
(32,)
Excel

## Classification using 5 bands and no-valences:
### Here it was classified combining the valences, bands and ROIs in a single dimension.

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------
N_Band=5
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],3)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,:,:]))
    F1_test_fold = np.zeros([N_folds,N_Band])
    ACC_test_fold = np.zeros([N_folds,N_Band])
    F1_test = np.zeros([N_Band])
    ACC_test = np.zeros([N_Band])
    STD_F1_test = np.zeros([N_Band])
    STD_ACC_test = np.zeros([N_Band])
    y_test = []
    
    print('Analizando Wavelet: '+wave_name[wave])

    fold = 0
    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        #print(emo,sub,fold,test_index)
        #print(ytest.shape,y_test.shape)
        y_test.append(ytest)

        for band in range(0,N_Band):        
            Xtrain,Xtest = np.squeeze(Features_Stats[band,train_index,:]), (Features_Stats[band,test_index,:])
            scaler = MinMaxScaler();
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            Xtrain = normalize(Xtrain)
            Xtest = normalize(Xtest)

            Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            Median= stats.median(Distances)
            Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
            Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
            SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
            SVM_Classifier.fit(Xtrain,ytrain)
            #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
            SVM_Prediction_test= SVM_Classifier.predict(Xtest)
            F1_test_fold[fold,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
            ACC_test_fold[fold,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
            #print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
       
            #print(y_test)
        fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
    print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    for band in range(0,N_Band):
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
        F1_test[band] = np.mean(F1_test_fold[:,band])
        ACC_test[band]= np.mean(ACC_test_fold[:,band])
        STD_F1_test[band] = np.std(F1_test_fold[:,band])
        STD_ACC_test[band] = np.std(ACC_test_fold[:,band])
        

    wav= wave_name[wave]+'_valencesrois'
    
    #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 
                
                

In [ ]:
print(F1_test_fold.shape)

## Classification using 8 Rois and no-valences:
### Here it was classified combining the valences, bands and ROIs in a single dimension.

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------
N_Band=8
CrossVal = KFold(n_splits=25,shuffle=False)
#C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
#N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],8)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,:,:]))
    F1_test_fold = np.zeros([N_folds,N_Band])
    ACC_test_fold = np.zeros([N_folds,N_Band])
    F1_test = np.zeros([N_Band])
    ACC_test = np.zeros([N_Band])
    STD_F1_test = np.zeros([N_Band])
    STD_ACC_test = np.zeros([N_Band])
    y_test = []
    
    print('Analizando Wavelet: '+wave_name[wave])

    fold = 0
    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        #print(emo,sub,fold,test_index)
        #print(ytest.shape,y_test.shape)
        y_test.append(ytest)

        for band in range(0,N_Band):        
            Xtrain,Xtest = np.squeeze(Features_Stats[band,train_index,:]), (Features_Stats[band,test_index,:])
            scaler = MinMaxScaler();
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            Xtrain = normalize(Xtrain)
            Xtest = normalize(Xtest)

            Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            Median= stats.median(Distances)
            Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
            Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
            SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
            SVM_Classifier.fit(Xtrain,ytrain)
            #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
            SVM_Prediction_test= SVM_Classifier.predict(Xtest)
            F1_test_fold[fold,band] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
            ACC_test_fold[fold,band]= acc(y_test[fold].T[0],SVM_Prediction_test)
            #print("y_test:",y_test[fold].T[0],"SVM:",SVM_Prediction_test,"ACC:",ACC_test_fold[fold],"F1:",F1_test_fold[fold],"C:",Parameters_grid['C'])
       
            #print(y_test)
        fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
    print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    for band in range(0,N_Band):
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
        F1_test[band] = np.mean(F1_test_fold[:,band])
        ACC_test[band]= np.mean(ACC_test_fold[:,band])
        STD_F1_test[band] = np.std(F1_test_fold[:,band])
        STD_ACC_test[band] = np.std(ACC_test_fold[:,band])
        

    wav= wave_name[wave]+'_valencesbands'
    
    #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 
                
                

# CODE FOR USE ALL FEATURES

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold as KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------
CrossVal = KFold(n_splits=25,shuffle=False)


#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],2)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[:,:]))
    F1_test_fold = np.zeros([N_folds])
    ACC_test_fold = np.zeros([N_folds])
    
    y_test = []
  
    
    print('Analizando Wavelet: '+wave_name[wave])

    fold = 0
    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[:,:]),label_matrix):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        #print(emo,sub,fold,test_index)
        #print(ytest.shape,y_test.shape)
        y_test.append(ytest)
     
        Xtrain,Xtest = Features_Stats[train_index,:], Features_Stats[test_index,:]
        scaler = MinMaxScaler();
        Xtrain = scaler.fit_transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        Xtrain = normalize(Xtrain)
        Xtest = normalize(Xtest)

        Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
        Median= stats.median(Distances)
        Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
        Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
        SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
        SVM_Classifier.fit(Xtrain,ytrain)
        #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
        SVM_Prediction_test= SVM_Classifier.predict(Xtest)
        F1_test_fold[fold] = f1_score(y_test[fold].T[0],SVM_Prediction_test,zero_division=1)
        ACC_test_fold[fold]= acc(y_test[fold].T[0],SVM_Prediction_test)
        print("FOLD:",fold,"\n ytest:",ytest," SVM_Prediction_test:",SVM_Prediction_test)

        fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
    print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])

    F1_test = np.mean(F1_test_fold[:])
    ACC_test= np.mean(ACC_test_fold[:])
    STD_F1_test = np.std(F1_test_fold[:])
    STD_ACC_test = np.std(ACC_test_fold[:])
        

    wav= wave_name[wave]+'_all'
    
     #Pickle save F1_Score
    path_save = "Wavelets_Results_ROIs_2021/"
    
    import pickle
    file= open(path_save+'F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open(path_save+'F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open(path_save+'Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open(path_save+'ACC_Score_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open(path_save+'ACC_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD
    file= open(path_save+'STD_F1_'+wav+'.pckl', 'wb')
    pickle.dump(STD_F1_test,file)
    file.close()

    File= open(path_save+'STD_F1_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
        
        
    #Pickle Save STD ACC
    file= open(path_save+'STD_ACC_'+wav+'.pckl', 'wb')
    pickle.dump(STD_ACC_test,file)
    file.close()

    File= open(path_save+'STD_ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == STD_ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 
                

                
                

In [ ]:
print("F1:",F1_test)
print("ACC:",ACC_test)
print("STD_F1:",STD_F1_test)
print("STD_ACC:",STD_ACC_test)

# CODE FOR USE ALL VALENCES VECTORIZED

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------
N_Band=5
CrossVal = KFold(n_splits=50,shuffle=False)
C = 2.**np.arange(-15,16)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix)
N_cs = len(C)

#---Iterations for each subject-------------------------------------------------------------------------------------------------
wave_name= ['bior3.3','bior6.8','sym2','sym5','sym8','coif1','coif5','db4','db10','db15']

for wave in range(4,5): 
    Features_Stats=characterization(EEG,num_valences,num_trials,num_channels,wave_name[wave],2)
    N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats[0,:,:]))
    F1_test_fold = np.zeros([N_folds,Valences,N_Band])
    ACC_test_fold = np.zeros([N_folds,Valences,N_Band])
    F1_test = np.zeros([Valences,N_Band])
    ACC_test = np.zeros([Valences,N_Band])
    STD_F1_test = np.zeros([Valences,N_Band])
    STD_ACC_test = np.zeros([Valences,N_Band])
    
    print('Analizando Wavelet: '+wave_name[wave])

    fold = 0
    for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats[1,:,:])):
        ytrain, ytest = label_matrix[train_index], label_matrix[test_index]
        #print(emo,sub,fold,test_index)
        #print(ytest.shape,y_test.shape)
        y_test.append(ytest)

        for band in range(0,N_Band):        
            Xtrain,Xtest = np.squeeze(Features_Stats[band,train_index,:]), (Features_Stats[band,test_index,:])
            scaler = MinMaxScaler();
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)

            Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
            Median= stats.median(Distances)
            Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
            Parameters_grid = svc_param_selection(Xtrain,ytrain,5,Gamma)
            SVM_Classifier= svm.SVC(kernel='rbf',C=Parameters_grid['C'],gamma=Gamma,class_weight = 'balanced')
            SVM_Classifier.fit(Xtrain,ytrain)
            #SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
            SVM_Prediction_test[fold,band]= SVM_Classifier.predict(Xtest)
            #print(y_test)
        fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
    print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
    for band in range(0,N_Band):
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band])
        F1_test[band] = f1_score(y_test,SVM_Prediction_test[:,band])
        ACC_test[band] = acc(y_test,SVM_Prediction_test[:,band])
    print(F1_test)
        

    wav= wave_name[wave]+'_valences'
    
    #Pickle save F1_Score
    import pickle
    file= open('Wavelets_Results_ROIs/F1_Score_'+wav+'.pckl', 'wb')
    pickle.dump(F1_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/F1_Score_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == F1_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")
    
    #Pickle save Features_Stats
    file= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wav+'.pckl', 'wb')
    pickle.dump(Features_Stats,file)
    file.close()

    File= open('Wavelets_Results_ROIs/Features_Stats_Wavelets_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == Features_Stats).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    #Pickle Save Accuracy
    file= open('Wavelets_Results_ROIs/ACC_'+wav+'.pckl', 'wb')
    pickle.dump(ACC_test,file)
    file.close()

    File= open('Wavelets_Results_ROIs/ACC_'+wav+'.pckl', 'rb')
    Load = pickle.load(File)
    File.close()
    print(Load.shape)
    if (Load == ACC_test).all:
        print("Excelente 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏")
    else:
        print("Sigue intentando")

    print('SE GUARDÓ EL ARCHIVO '+wav) 

                
                

In [ ]:
print(ACC_test)
mean=np.mean(ACC_test)
sd = np.std(ACC_test)
mean

In [ ]:
import warnings
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
warnings.filterwarnings('ignore')

#---Vars Definition-------------------------------------------------------------------------------------------------------------

Subjects = 32
N_Emociones=4
N_Band=6
CrossVal = LeaveOneOut()
N_folds = CrossVal.get_n_splits(np.squeeze(Features_Stats_Wavelets[0,0,:,:]))
print(N_folds)
C = 2.**np.arange(-15,19)
#SVM_C=Moda(N_Emociones,N_Band,Subjects,Best_C_Matrix_C3)
N_cs = len(C)
Best_C_Matrix=np.zeros([Subjects,N_Emociones,N_folds,N_Band])
F1_train = np.zeros([Subjects,N_Emociones,N_folds,N_Band,N_cs])
F1_test = np.zeros([Subjects,N_Emociones,N_Band])
SVM_Prediction_test=np.zeros([Subjects,N_Emociones,N_folds,N_Band])
y_test = np.zeros([Subjects,N_Emociones,N_folds])

#---Iterations for each subject-------------------------------------------------------------------------------------------------


for emo in range(N_Emociones):
    for sub in range(Subjects):
        print('Análisis Sujeto: ',sub)
        if sub==26 and emo==2:
            print(emo,' SALTO DE SUJETO ',sub)
            sub+=1
        emocion=emo
        fold = 0
        for train_index, test_index in CrossVal.split(np.squeeze(Features_Stats_Wavelets[sub,5,:,:])):
            ytrain, ytest = label_threshold_matrix_5[sub,train_index,emocion], label_threshold_matrix_5[sub,test_index,emocion]
            #print(emo,sub,fold,test_index)
            y_test[sub,emo,fold] = ytest
            for band in range(0,N_Band):
                Xtrain,Xtest = np.squeeze(Features_Stats_Wavelets[sub,band,train_index,:]), (Features_Stats_Wavelets[sub,band,test_index,:])
                Best_C= -10000
                Best_F1=0
                for c in range(len(C)):
                    Distances= scipy.spatial.distance.pdist(Xtrain, metric='euclidean')
                    Median= stats.median(Distances)
                    Gamma= 1/(Xtrain.shape[1]*Xtrain.var())
                    SVM_Classifier= svm.SVC(kernel='rbf',C=C[c],gamma=Gamma)
                    SVM_Classifier.fit(Xtrain,ytrain)
                    SVM_Prediction_train= SVM_Classifier.predict(Xtrain)
                    SVM_Prediction_test[sub,emo,fold,band]= SVM_Classifier.predict(Xtest)
                    F1_train[sub,emo,fold,band,c]= f1_score(ytrain,SVM_Prediction_train,average='macro')

                    if F1_train[sub,emo,fold,band,c]>Best_F1:
                        Best_C_Matrix[sub,emo,fold,band]=C[c]
                        Best_C= C[c]
                        Best_F1=F1_train[sub,emo,fold,band,c]
            fold+=1
        #---F1-score evaluartion for test set---------------------------------------------------------------------------------------              
        print('INICIO TEST')
        #Calcular F1_test
        #print(y_test[sub,:],SVM_Prediction_test[sub,:,band,c])
        for band in range(0,N_Band):
            F1_test[sub,emo,band] = f1_score(y_test[sub,emo,:],SVM_Prediction_test[sub,emo,:,band],average='macro')
        print(F1_test[sub])

                
                